# Exploring the Topology of Pegasus
D-Wave's newest quantum computer, Advantage, introduces a quantum processing unit (QPU) with a new architecture: the Pegasus family of topologies. This notebook explains the Pegasus topology and demonstrates its enhanced performance over QPUs of the previous generation used in the DW-2000Q system. 
    
1. [The Pegasus Advantage](#The-Pegasus-Advantage) shows the differences between the previous and new topologies and their usefulness.
2. [Navigating the Topology](#Navigating-the-Topology) demonstrates Ocean tools that help you use this QPU.
3. [Example Problem: RAN-K](#Example-Problem:-RAN-K) solves a hard problem on two generations of systems.

This notebook should familiarize you with the new Pegasus topology and the tools to use it.

<img src="images/anim.gif" width=200x/>

**New to Jupyter Notebooks?** JNs are divided into text or code cells. Pressing the **Run** button in the menu bar moves to the next cell. Code cells are marked by an "In: \[\]" to the left; when run, an asterisk displays until code completion: "In: \[\*\]".

# The Pegasus Advantage

The Advantage system is distinct from all previous generations of D-Wave quantum computers in the technological advance made by its new QPU architecture. 

The D-Wave QPU is a lattice of interconnected qubits. While some qubits connect to others via couplers, D-Wave QPUs are not fully connected. Instead, the qubits interconnect in an topology: Chimera for the 2000Q and Pegasus for the Advantage.

This layout of the D-Wave QPU is critical to translating a QUBO or Ising objective function into a format that a D-Wave system can solve.

## Minor-Embedding: Mapping a Problem to Qubits

<div class="alert alert-warning" role="alert" style="margin: 10px">Note: If you already understand how problems are mapped to the D-Wave system, please skip ahead to the next text cell.</div>
 
D-Wave systems solve binary quadratic models (BQM), the Ising model traditionally used in statistical mechanics and its computer-science equivalent, the quadratic unconstrained binary optimization (QUBO) problem. Given $N$ variables $x_1,...,x_N$, where each variable $x_i$ can have binary values $0$ or $1$, the system finds assignments of values that minimize

$\sum_i^N q_ix_i + \sum_{i<j}^N q_{i,j}x_i  x_j$,

where $q_i$ and $q_{i,j}$ are configurable (linear and quadratic) coefficients. 

Such objective functions can be represented by graphs. A graph comprises a collection of nodes (representing variables) and the connections between them (edges). For example, a Boolean AND, $z \Leftrightarrow x_1 \wedge x_2$, expressed in QUBO formulation as, 

$x_1 x_2 - 2(x_1+x_2)z +3z$, 

is represented by the graph:

<img src="images/embedding_and.png" width=300x/>

To formulate a problem for the D-Wave system is to program $q_i$ and $q_{i,j}$ so that assignments of $x_1,...,x_N$ also represent solutions to the problem. This requires that the problem graph be mapped to the QPU. [Minor embedding](#https://docs.ocean.dwavesys.com/en/stable/concepts/embedding.html#embedding-sdk) maps problem variables ($x_1, x_2, z$ for the AND gate) to the indexed qubits of the D-Wave QPU. On the D-Wave system, nodes are qubits and edges are couplers.

Were qubits on a QPU fully connected, with every qubit coupled to every other qubit, you could simply map each problem variable (graph node) to a qubit and each quadratic interaction (graph edge) to a coupler. But with sparser QPU topologies, a variable might be represented by a *chain* of two or more qubits. 

For example, the AND is represented by a $K_3$ fully connected graph (left graph in the figure below), and that cannot be mapped directly to a Chimera topology. Instead, a chain using two of four not-fully-connected qubits (the  middle graph below) is used to represent a single variable (right graph), here qubit 0 and qubit 4 to represent variable $z$.

<img src="images/embedding_chimera_and.png" width=500x/>

The strength of the coupler between qubits 0 and 4, which represents
variable $z$, is set to correlate the qubits strongly, so that in most
solutions they have a single value for $z$. 

Ocean software provides tools that handle minor-embedding. One such minor-embedding on a D-Wave 2000Q is displayed below using the Ocean software's [dwave-inspector](#https://docs.ocean.dwavesys.com/en/stable/docs_inspector/sdk_index.html) tool. The problem graph, shown on the left, is embedded in four qubits , shown on the right against a background of the Chimera topology. The variable highlighted in dark magenta is represented by two qubits, numbers 251 and 253 in this particular embedding.

<img src="images/embedding_3var4qubits.png" width=500x/>

The size and complexity of problems that can be submitted to a quantum computer depends on the QPU's *working graph*, the set of qubits and couplers that are available for computation. Adding qubits expands the QPU's range, obviously, but  denser connectivity reduces the wasting of qubits in representing single variables with chains of multiple qubits, which can also affect solution quality. 

The Pegasus topology enables the Advantage QPU to more than double the number of available qubits compared to the 2000Q, and the working graph is denser, meaning each qubit is coupled to a greater number of neighboring qubits. 

## More Qubits
This subsection looks at the straightforward increase in qubit numbers between the new and previous generation QPUs. 

It also introduces useful Ocean software tools that let you work with the QPU topology locally on your computer. 

In [ ]:
import dwave_networkx as dnx

C = dnx.chimera_graph(16)
P = dnx.pegasus_graph(16)

print("Qubits in full working graph: \n    Chimera: {} \n    Advantage: {}".format(len(C.nodes), len(P.nodes)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1, 2, figsize=(14,6))
dnx.draw_chimera(C, ax=ax[0], node_size=5, node_color='g')
dnx.draw_pegasus(P, ax=ax[1], node_size=5, node_color='b')
ax[0].title.set_text('Chimera C16')
ax[1].title.set_text('Pegasus P16')

Intuitively, you can scale up the size of problems embedded on a QPU more if they are sparse because these come closer to representing a string of variables connected head to tail by a similar string of qubits.

The next cells define a couple of functions used throughout this section:

* `generate_ran1` generates a problem with `variables` number of variables and `interactions` density of interactions.
* `try_embedding` tries to embed a problem in both the Chimera and Pegasus graphs. 

In [ ]:
import networkx as nx
import dimod 

def generate_ran1(variables, interactions, draw=True):
    
    G = nx.random_regular_graph(n=variables, d=interactions)
    bqm = dimod.generators.random.ran_r(1, G)
    
    if draw:
        plt.figure(figsize=(7, 7))
        nx.draw_networkx(G, pos=nx.spring_layout(G), with_labels=False, node_size=25) 
        plt.show()
            
    return bqm

bqm = generate_ran1(100, 2)

In [ ]:
import minorminer

def try_embedding(bqm, timeout=60, tries=2):
    
    max_len = {}
    for topology in [('Chimera', C), ('Pegasus', P)]:
        
        embedding = minorminer.find_embedding(bqm.quadratic, 
                                          topology[1].edges, 
                                          timeout=timeout, 
                                          tries=tries)
        if not embedding:
            print("{}: failed to embed.".format(topology[0]))
        else:
            max_len[topology[0]] = max([len(embedding[n]) for n in embedding])
            print("{}: found embedding with longest chain of {} qubits.".format(topology[0], max_len[topology[0]]))
            
    return max_len

answer = try_embedding(bqm, timeout=20, tries=2)

Try embedding problems with sizes that range from a quarter of the qubits in a full Chimera graph to nearly all the qubits. 

<div class="alert alert-warning" role="alert" style="margin: 10px">Note: In this and similar code cells below, you can shorten the runtime by setting a lower value to the `problems` parameter, representing the number of problems of each size, and/or the list different sizes, `variables`, set for the loop.</div>

In [ ]:
interactions = 2
problems = 2

draw_problem = True  # Change to false to not display problem graphs 

for variables in [500, 1000, 2000]:
        
    for problem in range(problems):
        
        print("\nProblem {} of {} for {} variables:".format(problem + 1, 
                                                            problems, 
                                                            variables))
        
        bqm = generate_ran1(variables, interactions, draw_problem)
        try_embedding(bqm)

In fact, you see that even for sparse problems, connectivity plays a crucial role in enabling you to embed large problems in a QPU. The power of the Pegasus topology comes from its higher connectivity. 

## Increased Connectivity

To properly appreciate the contribution of increasing the density of the QPU topology, increase the sparse problem's connectivity from 2 to 3:

In [ ]:
import pandas as pd

interactions = 3
problems = 2

draw_problem = True  # Change to false to not display problem graphs 

row = []
df_columns = ["Variables", "Problem", "Longest Chain"]

for variables in [100, 200, 400, 800]:
        
    for problem in range(problems):
        
        print("\nProblem {} of {} for {} variables:".format(problem + 1, 
                                                            problems, 
                                                            variables))
        
        bqm = generate_ran1(variables, interactions, draw_problem)
        
        row.append([variables, problem, try_embedding(bqm)])

results = pd.DataFrame(row, columns=df_columns)

In [ ]:
results

Note the fast growth of chain lengths even for such sparse problems. 

Try raising density for a relatively small, 100-variables problem:

In [ ]:
variables = 100
problems = 2

draw_problem = True  # Change to false to not display problem graphs 

row = []
df_columns = ["Interaction", "Problem", "Longest Chain"]

for interactions in range(2, 16, 2):
        
    for problem in range(problems):
        
        print("\nProblem {} of {} for {} interactions:".format(problem + 1, 
                                                            problems, 
                                                            interactions))
        
        bqm = generate_ran1(variables, interactions, draw_problem)
        
        row.append([interactions, problem, try_embedding(bqm)])

results = pd.DataFrame(row, columns=df_columns)

Plot the results:

In [ ]:
fig, ax = plt.subplots()
rects1 = ax.bar(results["Interaction"], results["Longest Chain"].apply(lambda x: x['Chimera']), 0.35, label='Chimera')
rects2 = ax.bar(results["Interaction"], results["Longest Chain"].apply(lambda x: x['Pegasus']), 0.35, label='Pegasus')

ax.set_ylabel('Longest Chain')
ax.set_xlabel('Interactions')
ax.set_title('Longest Chains for Each Topology')
ax.legend()

The preceding cells should have demonstrated two important points: 

* A topology with denser connectivity enables you to scale up your problems in terms of both the number of variables and the density of the variables' interactions.
* Minor-embedding heuristic algorithms can be slow. 

## Clique Embeddings
Clique embeddings can be very useful. A minor-embedding for a $K_n$ fully connected graph can be used for all minors of that graph. This means that if your application needs to submit a series of problems of up to size $n$ to the QPU, if you have an embedding for the $K_n$ graph on the QPU, you can simply reuse that embedding for all your problems.  

As an intuitive example, the $K_3$ minor-embedding explained above can be reused to embed a two-variable problem, $x_1x_2$ by not setting the values of variable $z$ and its couplers:

<img src="images/embedding_clique_k3_2vars.png" width=500x/>


<div class="alert alert-warning" role="alert" style="margin: 10px">Note that if your problems are sparse, using a clique embedding can be very wasteful, needlessly restricting the number of variables you can scale up to.</div>

### Embedding in a Single Chimera Unit-Cell ($K_{4,4}$)
Many problems have repetitive structures or can be formulated with repeated, small elements; for example, a problem expressed using Boolean elements such as the AND expression above. In such cases, it's advantageous to minor-embed the problem in a way that exploits the lattice structure of the QPU.  

The Chimera topology's simpler structure is based on a 16x16 grid of *unit cells*, each with four horizontal qubits connected to four vertical qubits via couplers. 

The figure below shows a 3x3 lattice of Chimera unit cells.
<img src="images/chimera.png" width=300x/>
If repeated structures of a problem can be embedded in a unit cell, you might then be able to repeat those structures across other unit cells in the lattice. 

Try minor-embedding cliques of varying sizes into a Chimera unit cell and an equivalent $K_{4,4}$ structure of the Pegasus topology, which is explained in the next section.

In [ ]:
C1 = dnx.chimera_graph(1)
P1 = dnx.pegasus_graph(2, node_list=[4, 5, 6, 7, 40, 41, 42, 43])

fig, ax = plt.subplots(1, 2, figsize=(14,6))
dnx.draw_chimera(C1, ax=ax[0], node_size=2000, with_labels=True, node_color='g')
dnx.draw_pegasus(P1, ax=ax[1], node_size=2000, with_labels=True, node_color='b')

In [ ]:
from dwave.embedding import *

for variables in range(2, 7):
    
    try:
        embedding = chimera.find_clique_embedding(variables, 1, target_edges=C1.edges)
        print("Chimera: embedded {} variables with longest chain of {}.".format(variables, max([len(chain) for chain in embedding.values()])))
    except ValueError:
        print("Chimera: embedding {} variables failed.".format(variables))

    try:
        #embedding = pegasus.find_clique_embedding(variables, target_graph=P1)
        embedding = minorminer.find_embedding(nx.complete_graph(variables), P1.edges)
        print("Pegasus: embedded {} variables with longest chain of {}.\n".format(variables, max([len(chain) for chain in embedding.values()])))
    except ValueError:
        print("Pegasus: embedding {} variables failed.\n".format(variables))

Below are [dwave-inspector](#https://docs.ocean.dwavesys.com/en/stable/docs_inspector/sdk_index.html) images of a $K_{5,5}$ clique embedded in the Chimera topology and a $K_{6,6}$ clique embedded in the Pegasus topology:

<img src="images/k_55_chimera.png" width=400x/>

<img src="images/k_66_pegasus.png" width=400x/>

### Largest Cliques

For a given maximum chain length, you can embed a clique of the following sizes in Pegasus and Chimera graphs: 

| Chain Length | 2 | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 | 11 | 12 | 13 | 14 |
|--------------|---|----|----|----|----|----|----|----|----|----|----|----|----|
| Chimera      | 4 | 8  | 12 | 16 | 20 | 24 | 28 | 32 | 36 | 40 | 44 | 48 | 52 |
| Pegasus      | 6 | 16 | 26 | 36 | 42 | 49 |    |    |    |    |    |    |    |

DRAFT CONTENT

In general, you can let Ocean automatically handle minor-embedding for you. But performance for some problems can be enhanced by intelligent manual adjustments. The next section explains the topology.   

# Navigating the Topology
You've seen the importance of connectivity. The connectivity of a QPU comes from its couplers. A 2000Q QPU has X couplers; an Advantage QPU has Y couplers. 

You have also seen the lattice structure of the QPU and an example of its importance for embedding your problem.

These two are aspects of the same thing: the lattice structure is determined by the couplings of its qubits.  

Chimera qubits are “oriented” on the QPU vertically or horizontally and coupled by one of two categories of couplers:

* *Internal couplers* connect pairs of orthogonal (with opposite orientation) qubits. Each qubit is connected via internal coupling to 4 other qubits.

* *External couplers* connect colinear pairs of qubits---pairs of parallel qubits in the same row or column.

In the figure below, green circles at the intersections of qubits signify internal couplers; external couplers, shown as connected blue circles, couple vertical qubits to adjacent vertical qubits and horizontal qubits to adjacent horizontal qubits. The green horizontal qubit in the center couples internally to four vertical qubits, bolded black, in its own unit cell and to the two blue horizontal qubits in adjacent unit cells. 

<img src="images/chimera_couplers.png" width=400x/>

Chimera qubits are characterized as having:

* Nominal length 4&mdash;each qubit is connected to 4 orthogonal qubits through internal couplers.
* Degree 6&mdash;each qubit is coupled to 6 different qubits.

Plot four Chimera unit cells and notice the couplings: internal couplers connecting vertical and horizontal qubits of each cell (short edges) and external couplers connecting similarly oriented qubits of different unit cells.

In [ ]:
%matplotlib inline
import dwave_networkx as dnx
import matplotlib.pyplot as plt

C = dnx.chimera_graph(2)
    
dnx.draw_chimera(C, with_labels=True, node_size=500, node_color='g')
plt.show()

When exploiting the underlying structure of the topology it can be helpful to use a coordinate system based on the structure. Qubits in the depictions above of unit cells are shown with the indexical label scheme, but for navigating, you might prefer coordinates based on the qubit position within a unit cell and the cell's place in the latticed.

For an m-by-n Chimera lattice, connections can be expressed using a node-indexing notation (i,j,u,k) for each node.

* (i,j) indexes the (row, column) of the Chimera tile. i must be between 0 and m-1, inclusive, and j must be between 0 and n-1, inclusive.
* u=0 indicates the left-hand nodes in the tile, and u=1 indicates the right-hand nodes.
* k=0,1,…,t-1 indexes nodes within either the left- or right-hand shores of a tile.

For the C2 Chimera lattice above show the *Chimera coordinate* of the second vertical and horizontal qubit in each cell:

In [ ]:
for i in range(1, 32, 4):
    print("qubit {} has Chimera coordinates {}".format(i, C.nodes(data=True)[i]['chimera_index']))

Ocean utilities are available to translate between coordinates.

In [ ]:
coords = dnx.chimera_coordinates(2)

i = 13
c = (1, 1, 0, 1)

print("Qubit {} has Chimera coordinates {}.".format(i, coords.linear_to_chimera(i)))
print("Chimera coordinates {} designate qubit {}.".format(c, coords.chimera_to_linear(c)))

Pegasus qubits are also oriented on the QPU vertically or horizontally but they are coupled by one of three categories of couplers:

* Internal couplers connect pairs of orthogonal (with opposite orientation) qubits. Each qubit is connected via internal coupling to 12 other qubits.

  In comparison with Chimera, internal coupling in Pegasus connects each qubit
  to qubits of opposite orientation in repeated substructures that include more
  than a the internal couplings of a single Chimera unit cell (a $K_{4,4}$
  bipartite graph.

* External couplers connect vertical qubits to adjacent vertical qubits and horizontal qubits to adjacent horizontal qubits.

* *Odd couplers* connect similarly aligned pairs of qubit sin the same Chimera unit cell, a vertical qubit to another vertical qubit and a horizontal qubit to another horizontal qubit.

The figure below provides a helpful way to envision a recurring structure of the Pegasus topology, similar to the unit cells of Chimera: the division of
internal couplings into $K_{4,4}$ bipartite graphs abstracted as three layers of
Chimera lattices. In this abstraction, each qubit forms part, through its
internal couplers, of a Chimera unit cell in one layer (translucent green square) while
additionally coupling to four qubits of a unit cell in a second layer (translucent blue square)
and two qubits each of two units cells in a third layer (translucent pink squares).

<img src="images/pegasus_zlayered_unitcells.png" width=400x/>

In [ ]:
P = dnx.pegasus_graph(2)

for i in range(2, 14):
    print("Qubit {} has Pegasus coordinates {}.".format(i, P.nodes(data=True)[i]['pegasus_index']))

H = dnx.pegasus_graph(2, node_list=[node for node in range(2, 14)])

fig, ax = plt.subplots(1, 1, figsize=(8,8))
dnx.draw_pegasus(P, ax=ax, with_labels=True, node_size=500, node_color='b')
dnx.draw_pegasus(H, ax=ax, node_color='g')

# Example Problem: RAN-K

## Solver Availability
This subsection checks whether you have access to both generations of solvers

In [ ]:
import os

from dwave.system.samplers import DWaveSampler
from dwave.cloud.exceptions import *

try:
    qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
    qpu_2000q = DWaveSampler(solver={'topology__type': 'chimera', 'qpu': True})
    
    qpus = {'advantage': qpu_advantage, '2000q': qpu_2000q}

    print("Connected to Advantage {} and 2000Q {}.".format(qpu_advantage.solver.id, qpu_2000q.solver.id))
except SolverNotFoundError:
    print("Currently a pair of solvers are unavailable for sections comparing QPU technologies. Try those examples later.")


# Example Problem: Graph Partitioning 
There a 

## Formulating the Problem


### Sparse Graphs

In [ ]:
##########################
import random
import networkx as nx

num_clusters = 20
clusters = [random.randint(2, 3) for x in range(num_clusters)]
G = nx.random_partition_graph(sizes=clusters, p_in=0.2, p_out=0.05)

In [ ]:
import networkx as nx

graph_nodes = 10
cluster_size = 3
density = 0.5
density_external = 0.05
G = nx.planted_partition_graph(l=graph_nodes, k=cluster_size, p_in=density, p_out=density_external)

In [ ]:
G = nx.LFR_benchmark_graph(n=40, 
                           tau1=5, 
                           tau2=4, 
                           mu=0.2, 
                           average_degree=3,  
                           max_degree=5, 
                           min_community=10, 
                           max_community=15)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 7))
nx.draw_networkx(G, pos=nx.spring_layout(G)) 
plt.show()

### Dense Graphs

In [ ]:
#####################################
import networkx as nx

graph_nodes = 10
G = nx.complete_graph(graph_nodes)

density = 1
gamma = 0.5*(0.5*graph_nodes*(graph_nodes - 1))*density
chain_strength = gamma*graph_nodes

In [ ]:
import networkx as nx

graph_nodes = 40
density = 1
G = nx.erdos_renyi_graph(n=graph_nodes, p=density)

gamma = 0.5*(0.5*graph_nodes*(graph_nodes - 1))*density
chain_strength = gamma*graph_nodes

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 7))
nx.draw_networkx(G, pos=nx.spring_layout(G)) 
plt.show()

## Test Ising Problem

In [ ]:
import networkx as nx

nodes = ['a', 'b', 'c']
edges = [('a', 'b'), ('b', 'c')]

h = {'a': 2, 'b': 0, 'c': -2}
J = {edge: 1.5 for edge in edges}

print(h, J)

In [ ]:
best_embedding = {'advantage': {'a': [336], 
                                'b': [479, 321], 
                                'c': [524]},
                 '2000q': {'a': [1445], 'b': [1447, 1443], 'c': [1441]}}

In [ ]:
from dwave.system import FixedEmbeddingComposite
samplers = {qpu: FixedEmbeddingComposite(qpus[qpu], best_embedding[qpu]) for qpu in qpus}

samplesets = {}
for qpu in qpus:  
    samplesets[qpu] = samplers[qpu].sample_ising(h, J,
                                           num_reads=num_reads, 
                                           auto_scale=True,
                                           chain_strength=1.5)
    
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))
    print(samplesets[qpu])

In [ ]:
def scaled_ising(h, J, factor=1.0):
    range_h = [min(h[key] for key in h.keys()), 
               max(h[key] for key in h.keys())]
    range_J = [min(J[key] for key in J.keys()), 
               max(J[key] for key in J.keys())]
    
    scaling_h = max(range_h[0]/-1.0, range_h[1]/1.0)
    scaling_J = max(range_J[0]/-1.0, range_J[1]/1.0)
    scaling = max(scaling_h, scaling_J)
    return dict((key, factor*h[key]/scaling) for key in h.keys()), dict((key, factor*J[key]/scaling) for key in J.keys())

In [ ]:
h_scaled, J_scaled =  scaled_ising(h, J, factor=1.0)

range_hs = [min(h_scaled[key] for key in h_scaled.keys()), 
               max(h_scaled[key] for key in h_scaled.keys())]
range_Js = [min(J_scaled[key] for key in J_scaled.keys()), 
               max(J_scaled[key] for key in J_scaled.keys())]

print("Range h {}".format(range_hs))
print("Range J {}".format(range_Js))


In [ ]:
from dwave.embedding import embed_ising

chain_strength_scaled = 0.75

embedded_vals = {}
for qpu in qpus: 
    embedded_vals[qpu] = embed_ising(h_scaled, 
                                          J_scaled, 
                                          best_embedding[qpu], 
                                          qpus[qpu].adjacency, 
                                          chain_strength=chain_strength_scaled)
    
    print("{} J: {}".format(qpu, sorted(set(embedded_vals[qpu][1][key] 
                            for key in embedded_vals[qpu][1].keys()))))    

In [ ]:
plt.figure(figsize=(6, 8))
axis = 1
for qpu_name, qpu in qpus.items(): 
    ax = "ax"+str(axis)
    ax = plt.subplot(2, 1, axis)
    ax.set_title("Sampler: " + qpu_name)
    ax.set_xlabel("--")
    ax.set_ylabel("J")
    ax.plot(range(len(embedded_vals[qpu_name][1])), embedded_vals[qpu_name][1].values(), 'bo')
    axis += 1


In [ ]:
samplesets_embedded = {}
for qpu in qpus:  
    samplesets_embedded[qpu] = qpus[qpu].sample_ising(embedded_vals[qpu][0], 
                                           embedded_vals[qpu][1], 
                                           num_reads=num_reads,
                                           answer_mode='histogram',
                                           auto_scale=False)

In [ ]:
from dwave.embedding import unembed_sampleset, chain_break_frequency

samplesets = {}
for qpu in qpus:  
        
    samplesets[qpu] = unembed_sampleset(target_sampleset=samplesets_embedded[qpu], 
                                        embedding=best_embedding[qpu], 
                                        source_bqm=dimod.BinaryQuadraticModel.from_ising(h, J), 
                                        chain_break_method=None, 
                                        chain_break_fraction=True, 
                                        return_embedding=True)
    
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))
    print(samplesets[qpu])



### Graph --> QUBO

In [ ]:
from collections import defaultdict
from itertools import combinations


Q = defaultdict(int)

# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1
    Q[(v,v)] += 1
    Q[(u,v)] += -2

for i in G.nodes:
    Q[(i,i)] += gamma*(1-len(G.nodes))

for i, j in combinations(G.nodes, 2):
    Q[(i,j)] += 2*gamma

## Submitting for Solution

### Minor Embedding

In [ ]:
from collections import defaultdict
import minorminer
import numpy as np

embedding_tries = 1

best_embedding = defaultdict(lambda: ({}, 1000))
for qpu in qpus:
    print("\nFinding a good embedding for {}:\n".format(qpu))
    for i in range(embedding_tries):
        embedding = minorminer.find_embedding(Q.keys(), qpus[qpu].edgelist, timeout=60, tries=1)
        if not embedding:
            print("   Failed to find an embedding {}.".format(i))
        else:
            chain_lengths = list(len(chain) for chain in embedding.values())
            avg_chain = np.average(chain_lengths)
            print("    Embedding {} chain lengths: {:.2f} average, {:.2f} std, {} max, {} min.".
                  format(i,
                         avg_chain,
                         np.std(chain_lengths),
                         np.max(chain_lengths),
                         np.min(chain_lengths)))
            if avg_chain < best_embedding[qpu][1]:
                best_embedding[qpu] = (embedding, avg_chain) 
    print("Best for {} has average length {}.\n".format(qpu, best_embedding[qpu][1]))


### Using Virtual Graph

In [ ]:
def scaled_qubo(Q, factor=1.0):
    range_linear = [min(Q[key] for key in Q.keys() if key[0] == key[1]), 
                max(Q[key] for key in Q.keys() if key[0] == key[1])]
    range_quadratic = [min(Q[key] for key in Q.keys() if key[0] != key[1]), 
                   max(Q[key] for key in Q.keys() if key[0] != key[1])]
    
    scaling_linear = max(range_linear[0]/-1.0, range_linear[1]/1.0)
    scaling_quadratic = max(range_quadratic[0]/-1.0, range_quadratic[1]/1.0)
    scaling = max(scaling_linear, scaling_quadratic)
    return dict((key, factor*Q[key]/scaling) for key in Q.keys())

for qpu in qpus:
    print("Coupler strength range:", qpus[qpu].properties['extended_j_range'])

In [ ]:
num_reads = 1000
t_anneal = 1
vg_chain_strength = 1.0
scaling = 2

In [ ]:
from dwave.system import VirtualGraphComposite

samplers = {"sampler_" + str(qpu): VirtualGraphComposite(qpus[qpu], 
            best_embedding[qpu][0], 
            chain_strength=vg_chain_strength) 
            for qpu in qpus}

samplesets = {}
for qpu, sampler in samplers.items():  
    samplesets[qpu] = sampler.sample_qubo(scaled_qubo(Q, scaling), 
                                          num_reads=num_reads, 
                                          annealing_time=t_anneal, 
                                          answer_mode='histogram', 
                                          apply_flux_bias_offsets=True)

    partition = [sum(samplesets[qpu].first.sample.values()), graph_nodes - sum(samplesets[qpu].first.sample.values())]
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Partition for {} is {} with lowest energy {:.3f}.".format(qpu, partition, samplesets[qpu].first.energy))
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 3*len(samplesets)))
axis = 1
for qpu, sampler in samplers.items(): 
    ax = "ax"+str(axis)
    ax = plt.subplot(len(samplesets)+1, 1, axis)
    ax.set_title("Sampler: " + qpu)
    ax.set_xlabel("Sample")
    ax.set_ylabel("Energy")
    ax.plot(range(len(samplesets[qpu])), samplesets[qpu].record.energy, 'bo')
    axis += 1

lowest = round(0.1*len(samplesets["sampler_2000q"]))
ax = "ax"+str(axis)
ax = plt.subplot(len(samplesets)+1, 1, axis)
ax.set_title("Delta Energy")
ax.set_xlabel("Sample")
ax.set_ylabel("Delta Energy")
ax.plot(range(lowest), samplesets["sampler_2000q"].record.energy[0:lowest]-samplesets["sampler_advantage"].record.energy[0:lowest], 'bo')

In [ ]:
vg_flux_on = samplesets

### Direct Embedding

In [ ]:
from dwave.embedding import embed_qubo
embedded_Q = embed_qubo(Q_scaled, best_embedding['2000q'][0], qpus['2000q'].adjacency, chain_strength=1.0)
embedded_Q


### Fixed Embedding

In [ ]:
chain_strength = 4000
t_anneal = 1

In [ ]:
from dwave.system import FixedEmbeddingComposite

num_reads = 1000
t_anneal = 200

samplers = {"sampler_" + str(qpu): FixedEmbeddingComposite(qpus[qpu], best_embedding[qpu][0]) 
            for qpu in qpus}

samplesets = {}
for qpu, sampler in samplers.items():  
    samplesets[qpu] = sampler.sample_qubo(Q, num_reads=num_reads, 
                                             chain_strength=chain_strength, 
                                             annealing_time=t_anneal, 
                                             answer_mode='histogram')

    partition = [sum(samplesets[qpu].first.sample.values()), graph_nodes - sum(samplesets[qpu].first.sample.values())]
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Partition for {} is {} with lowest energy {:.3f}.".format(qpu, partition, samplesets[qpu].first.energy))
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 3*len(samplesets)))
axis = 1
for qpu, sampler in samplers.items(): 
    ax = "ax"+str(axis)
    ax = plt.subplot(len(samplesets)+1, 1, axis)
    ax.set_title("Sampler: " + qpu)
    ax.set_xlabel("Sample")
    ax.set_ylabel("Energy")
    ax.plot(range(len(samplesets[qpu])), samplesets[qpu].record.energy, 'bo')
    axis += 1

lowest = round(0.1*len(samplesets["sampler_2000q"]))
ax = "ax"+str(axis)
ax = plt.subplot(len(samplesets)+1, 1, axis)
ax.set_title("Delta Energy")
ax.set_xlabel("Sample")
ax.set_ylabel("Delta Energy")
ax.plot(range(lowest), samplesets["sampler_2000q"].record.energy[0:lowest]-samplesets["sampler_advantage"].record.energy[0:lowest], 'bo')

## Naive Submission

In [ ]:
chain_strength = 4000

In [ ]:
import numpy as np
from dwave.system import EmbeddingComposite

# Import the problem inspector to begin data capture
import dwave.inspector

num_reads = 1000

samplers = {"sampler_" + str(qpu): EmbeddingComposite(qpus[qpu]) for qpu in qpus}

samplesets = {}
for qpu, sampler in samplers.items():  
    samplesets[qpu] = sampler.sample_qubo(Q, num_reads=num_reads,
                                             answer_mode='raw',
                                             chain_strength=chain_strength)
    
    partition = [sum(samplesets[qpu].first.sample.values()), graph_nodes - sum(samplesets[qpu].first.sample.values())]
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Partition for {} is {} with lowest energy {:.3f}.".format(qpu, partition, samplesets[qpu].first.energy))
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 3*len(samplesets)))
axis = 1
for qpu, sampler in samplers.items(): 
    ax = "ax"+str(axis)
    ax = plt.subplot(len(samplesets)+1, 1, axis)
    ax.set_title("Sampler: " + qpu)
    ax.set_xlabel("Sample")
    ax.set_ylabel("Energy")
    ax.plot(range(len(samplesets[qpu])), samplesets[qpu].record.energy, 'bo')
    axis += 1

lowest = round(0.1*len(samplesets["sampler_2000q"]))
ax = "ax"+str(axis)
ax = plt.subplot(len(samplesets)+1, 1, axis)
ax.set_title("Delta Energy")
ax.set_xlabel("Sample")
ax.set_ylabel("Delta Energy")
ax.plot(range(lowest), samplesets["sampler_2000q"].record.energy[0:lowest]-samplesets["sampler_advantage"].record.energy[0:lowest], 'bo')

## Hamming Distance

In [ ]:
import numpy as np
def get_hamming_distance(x1, x2):    # x1, x2 are NumPy arrays
    return np.sum(x1 != x2)

def normalized_hamming_distance(sols):
    sols = np.array(sols)
    hd = np.true_divide(np.array([get_hamming_distance(x1, x2) for x1, x2 in zip(sols, sols[1:])]), np.shape(sols)[1])
    return [hd, np.mean(hd)]


In [ ]:
samplesets = {}
for qpu, sampler in samplers.items():  
    samplesets[qpu] = sampler.sample_qubo(Q, num_reads=num_reads, 
                                             chain_strength=chain_strength, 
                                             annealing_time=t_anneal, 
                                             answer_mode='raw')

for sampler in samplers.keys():
    print("Lowest energy for {} is {:.3f}.".format(sampler, samplesets[sampler].first.energy))

In [ ]:
from helpers.draw import plot_hamming # To see helper functions, select Jupyter File Explorer View from the Online Learning page

hamming_distance = {}
for qpu, sampler in samplers.items():  
    hamming_distance[qpu] = normalized_hamming_distance(samplesets[qpu].record.sample)

print("QPU time used: {} microseconds.".format(sum(samplesets[qpu].info['timing']['qpu_access_time'] for qpu in samplers)))
plot_hamming(hamming_distance)


## BEST SUBMISSION

In [ ]:
max(x for x in Q.values())

In [ ]:
# VERIFY QUBO
from dwave.system import EmbeddingComposite

chain_strength = 1000
num_reads = 1000

samplers = {"sampler_" + str(qpu): EmbeddingComposite(qpus[qpu]) for qpu in qpus}

samplesets = {}
for qpu, sampler in samplers.items():  
    samplesets[qpu] = sampler.sample_qubo(Q, num_reads=num_reads, 
                                             chain_strength=chain_strength)
    
    partition = [sum(samplesets[qpu].first.sample.values()), graph_nodes - sum(samplesets[qpu].first.sample.values())]
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Partition for {} is {} with lowest energy {:.3f}.".format(qpu, partition, samplesets[qpu].first.energy))
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))

In [ ]:
import dimod
(h, J, offset) = dimod.qubo_to_ising(Q)

range_h = [min(h[key] for key in h.keys()), 
               max(h[key] for key in h.keys())]
range_J = [min(J[key] for key in J.keys()), 
               max(J[key] for key in J.keys())]

print("Range h {}".format(range_h))
print("Range J {}".format(range_J))

In [ ]:
# QUBO --> ISING
samplers = {"sampler_" + str(qpu): EmbeddingComposite(qpus[qpu]) for qpu in qpus}

samplesets = {}
for qpu, sampler in samplers.items():  
    samplesets[qpu] = sampler.sample_ising(h, J, 
                                           num_reads=num_reads, 
                                           auto_scale=True,
                                           chain_strength=4000)
    
    partition_miss = sum(samplesets[qpu].first.sample.values())
    avg_breaks = 100*sum(samplesets[qpu].record.chain_break_fraction)/num_reads
    print("Partition miss for {} is {} with lowest energy {:.3f}.".format(qpu, partition_miss, samplesets[qpu].first.energy))
    print("Chain breaks for {} is {:.3f} percent.".format(qpu, avg_breaks))

In [ ]:
def scaled_ising(h, J, factor=1.0):
    range_h = [min(h[key] for key in h.keys()), 
               max(h[key] for key in h.keys())]
    range_J = [min(J[key] for key in J.keys()), 
               max(J[key] for key in J.keys())]
    
    scaling_h = max(range_h[0]/-1.0, range_h[1]/1.0)
    scaling_J = max(range_J[0]/-1.0, range_J[1]/1.0)
    scaling = max(scaling_h, scaling_J)
    return dict((key, factor*h[key]/scaling) for key in h.keys()), dict((key, factor*J[key]/scaling) for key in J.keys())

In [ ]:
h_scaled, J_scaled =  scaled_ising(h, J, factor=0.1)

range_hs = [min(h_scaled[key] for key in h_scaled.keys()), 
               max(h_scaled[key] for key in h_scaled.keys())]
range_Js = [min(J_scaled[key] for key in J_scaled.keys()), 
               max(J_scaled[key] for key in J_scaled.keys())]

print("Range h {}".format(range_hs))
print("Range J {}".format(range_Js))


In [ ]:
from dwave.embedding import embed_ising

chain_strength_scaled = 2.

embedded_vals = {}
for qpu in qpus: 
    embedded_vals[qpu] = embed_ising(h_scaled, 
                                          J_scaled, 
                                          best_embedding[qpu][0], 
                                          qpus[qpu].adjacency, 
                                          chain_strength=chain_strength_scaled)
    
    print("{} J: {}".format(qpu, sorted(set(embedded_vals[qpu][1][key] 
                            for key in embedded_vals[qpu][1].keys()))))    

In [ ]:
plt.figure(figsize=(6, 8))
axis = 1
for qpu_name, qpu in qpus.items(): 
    ax = "ax"+str(axis)
    ax = plt.subplot(2, 1, axis)
    ax.set_title("Sampler: " + qpu_name)
    ax.set_xlabel("--")
    ax.set_ylabel("J")
    ax.plot(range(len(embedded_vals[qpu_name][1])), embedded_vals[qpu_name][1].values(), 'bo')
    axis += 1


In [ ]:
from dwave.embedding import unembed_sampleset, chain_break_frequency

num_reads = 5000

samplesets_embedded = {}
samplesets = {}
chain_breaks = {}

for qpu in qpus:  
    samplesets_embedded[qpu] = qpus[qpu].sample_ising(embedded_vals[qpu][0], 
                                           embedded_vals[qpu][1], 
                                           num_reads=num_reads,
                                           answer_mode='histogram',
                                           auto_scale=False)
    samplesets[qpu] = unembed_sampleset(target_sampleset=samplesets_embedded[qpu], 
                                        embedding=best_embedding[qpu][0], 
                                        source_bqm=dimod.BinaryQuadraticModel.from_ising(h, J), 
                                        chain_break_method=None, 
                                        chain_break_fraction=True, 
                                        return_embedding=True)
    
    partition_miss = sum(samplesets[qpu].first.sample.values())
    print("Partition miss for {} is {} with lowest energy {:.3f}.".format(qpu, partition_miss, samplesets[qpu].first.energy))

    chain_breaks[qpu] = chain_break_frequency(samplesets_embedded[qpu], 
                                                          best_embedding[qpu][0])
    print(qpu, ":", {key:100*val for key, val in chain_breaks[qpu].items()})            


In [ ]:
samplesets["advantage"].record.chain_break_fraction

In [ ]:
from __future__ import division, absolute_import

import dimod
import numpy as np

from six import iteritems

from dwave.embedding.chain_breaks import broken_chains


def my_chain_break_frequency(samples_like, embedding):
    """Determine the frequency of chain breaks in the given samples.

    Args:
        samples_like (samples_like/:obj:`dimod.SampleSet`):
            A collection of raw samples. 'samples_like' is an extension of NumPy's array_like.
            See :func:`dimod.as_samples`.

        embedding (dict):
            Mapping from source graph to target graph as a dict of form {s: {t, ...}, ...},
            where s is a source-model variable and t is a target-model variable.

    Returns:
        dict: Frequency of chain breaks as a dict in the form {s: f, ...},  where s
        is a variable in the source graph and float f the fraction
        of broken chains.

    Examples:
        This example embeds a single source node, 'a', as a chain of two target nodes (0, 1)
        and uses :func:`.chain_break_frequency` to show that out of two synthetic samples,
        one ([-1, +1]) represents a broken chain.

        >>> import numpy as np
        ...
        >>> samples = np.array([[-1, +1], [+1, +1]])
        >>> embedding = {'a': {0, 1}}
        >>> print(dwave.embedding.chain_break_frequency(samples, embedding)['a'])
        0.5


    """
    if isinstance(samples_like, dimod.SampleSet):
        labels = samples_like.variables
        samples = samples_like.record.sample
        num_occurrences = samples_like.record.num_occurrences
    else:
        samples, labels = dimod.as_samples(samples_like)
        num_occurrences = np.ones(samples.shape[0])

    if not all(v == idx for idx, v in enumerate(labels)):
        labels_to_idx = {v: idx for idx, v in enumerate(labels)}
        embedding = {v: {labels_to_idx[u] for u in chain} for v, chain in embedding.items()}

    if not embedding:
        return {}

    variables, chains = zip(*embedding.items())

    broken = broken_chains(samples, chains)

    return {v: float(np.average(broken[:, cidx], weights=num_occurrences))
            for cidx, v in enumerate(variables)}



Copyright &copy; D-Wave Systems Inc.